In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from core.experiment import Experiment, ExpRandomeLabeledData
from copy import deepcopy
from types import MethodType
DEVICE = 'cuda:0'

In [3]:
print(Experiment.args_template())

{ 
    'output_root_dir' : , 
    'num_batches' : , 
    'tag' : , 
    'eval_interval' : , 
    'num_runs' : , 
    'num_samples' : , 
    'model' : , 
    'lr_init' : , 
    'weight_decay' : , 
    'ds_train' : , 
    'ds_test' : , 
    'momentum' : , 
    'augment' : , 
    'batch_size' : , 
    'losses' : , 
    'losses_track_only' : , 
    'w_losses' : , 
    'evaluation_policies'
}


In [8]:
args = \
{ 
    'output_root_dir' : '/tmp/debug/', 
    'num_batches' : 10000, 
    'tag' : 'normed_linear', 
    'eval_interval' : None, 
    'num_runs' : 1, 
    'num_samples' : None, 
    'model' : ("ResNet18", {
        'compactification': 'sphere_l2_learned', 'latent_dim': None, 'linear_bias': False, 
        'linear_cfg': ('NormedLinear', {'learn_radius': True})   
    }), 
    'lr_init' : 0.1, 
    'weight_decay' : 0.001, 
    'ds_train' : 'cifar10_train', 
    'ds_test' : 'cifar10_test', 
    'momentum' : 0.9, 
    'augment' : 'none', 
    'batch_size' : 256, 
    'losses' : (
        ('CrossEntropy', {'reduction': 'mean'}), 
#           ('SupConLoss', {'temperature': 0.1}),       
#           ('SupConLossWeighted', {'temperature': 0.1, 'weight': 0.}),    
    ), 
    'losses_track_only' : (), 
    'w_losses' : None, 
    'evaluation_policies': ('linear', 'retrained_linear', 'explicit_linear')
}

In [ ]:
def post_batch_hook(self):
    if self.batch_i % 10 == 0:
#         norm_w = torch.norm(self.model.cls.weight.data, p=2, dim=1)
#         self.logger.log_value('norm_w', norm_w)  
        self.logger.log_value('r_projection', self.model.feat_ext[-1]._r.data)
        self.logger.log_value('r_linear', self.model.cls._r.data)
    

with torch.cuda.device(DEVICE):
    exp = Experiment(**args)
    exp.post_batch = MethodType(post_batch_hook, exp)
    exp()

In [6]:
# with torch.cuda.device(DEVICE):
#     for bs in [8, 12]:
#         args_i = deepcopy(args)
#         args_i['batch_size'] = bs
#         exp = ExpRandomeLabeledData(**args_i)
#         exp()

In [7]:
5

5